# 痞客邦資料分析

## Preparation

In [69]:
# module import
import re
import json
import jieba
from bs4 import BeautifulSoup
from pprint import pprint

In [10]:
# the file path of pixnet
data_file_path = "./2017-pixnet-hackathon-data/article/food.json"

# the file path to our data
our_data_file_path = "./data.json"

## Load the json file

In [11]:
data = []
with open(data_file_path) as data_file:
    for line in data_file:
        data.append(json.loads(line))

In [12]:
print "Number of blogs: {}".format(len(data))

Number of blogs: 9635


In [13]:
# filter out data without content
data = [d for d in data if 'content' in d]


## Parse the phone

In [40]:
phone_pattern1 = r"電話.{,200}"
phone_pattern2 = r"電話.{,4}?([0-9\-\(][0-9\-\(\)\s]*[0-9])"
#phone_format = r"([0-9]+)[^0-9]*([0-9]*)[^0-9]*([0-9]*)[^0-9]*([0-9]*)"
#debug_pattern = "電話.{50}"

In [41]:
def get_phone(json_blog):
    result = []
    s_contain_phone = re.findall(phone_pattern1, json_blog['content'].encode('utf-8'))
    for scp in s_contain_phone:
        s_without_html = BeautifulSoup(scp).text
        s_phone = re.findall(phone_pattern2, s_without_html.encode('utf-8'))
        for sp in s_phone:
            result.append(sp)
    return result

In [46]:
# pattern refine process
count = 0
for d in data:
    phone = get_phone(d)
    if phone:
        count += 1
print "get {} phone".format(count)

get 5646 phone


## Parse the address

In [43]:
address_pattern1 = r"地址.{,200}"
address_pattern2 = r"地址：*\s*(.*(號|樓)\s*\)*)"

In [44]:
def get_address(json_blog):
    result = []
    s_contain_addr = re.findall(address_pattern1, json_blog['content'].encode('utf-8'))
    for sca in s_contain_addr:
        s_without_html = BeautifulSoup(sca).text
        s_addr = re.findall(address_pattern2, s_without_html.encode('utf-8'))
        for sa in s_addr:
            result.append(sa[0])
    return result

In [45]:
count = 0
for d in data:
    addresses = get_address(d)
    if addresses:
        count += 1
print "get {} address".format(count)

get 4399 address


## Parse the time

In [59]:
time_pattern1 = r"營業時間.{,200}"

In [68]:
for d in data:
    s_contain_time = re.findall(time_pattern1, d['content'].encode('utf-8'))
    for s in s_contain_time:
        pass
        #print s

## Write json

In [47]:
our_data = {}
for d in data:
    id = d['article_id']
    our_data[id] = {}
    our_data[id]['url'] = d['url']
    our_data[id]['hits'] = d['hits']
    
    # get address
    address = get_address(d)
    phone = get_phone(d)
    
    if address:
        our_data[id]['address'] = address[-1]
    if phone:
        our_data[id]['phone'] = phone[-1]
    if address and phone:
        our_data[id]['usable'] = True
    else:
        our_data[id]['usable'] = False

In [48]:
with open(our_data_file_path, 'w') as data_file:
    json.dump(our_data, data_file, ensure_ascii=False)

In [53]:
sum([1 for _, d in our_data.items() if d['usable']])

3705

In [66]:
for _, d in our_data.items():
    if 'address' in d:
        print d['address']
        print d['url']
        break

台北市大安區敦化南路一段160巷13號1樓 
http://tony60533.pixnet.net/blog/post/30745555


In [63]:
our_data['99647661']

{'hits': 1601287,
 'url': u'http://nigi33kimo.pixnet.net/blog/post/99647661',
 'usable': False}